In [ ]:
from pathlib import Path

import geopandas as gpd
import icechunk
import numpy as np
import pandas as pd
import xarray as xr
import zarr
from icechunk.xarray import to_icechunk
from tqdm import tqdm
from virtualizarr import open_virtual_dataset

In [3]:
import boto3
import os

profile_name = "CIROH_USER"  # Replace with your AWS credentials file profile name
os.environ['AWS_PROFILE'] = profile_name

In [4]:
file_path = Path("/Users/taddbindas/data/dhbv_hf/HydroFabric_forward_1980_2019_From_dPL_local_daymet_v6_2v18_2_oneGPU_dynamic_k0_1980_1995")
bucket="mhpi-spatial"
prefix="hydrofabric_v2.2_dhbv_retrospective"
commit="initial streamflow commit"

In [6]:
if file_path.exists() is False:
    raise FileNotFoundError(f"Cannot find: {file_path}")
root = zarr.open_group(file_path)
# root.tree()

### Need to do the following
- get attributes/catchment area from hydrofabric
- get sorting mechanism
- implement the code below
- write to icechunk

In [13]:
gdf = gpd.read_file("/Users/taddbindas/hydrofabric/v2.2/conus_nextgen.gpkg", layer="flowpaths")
gdf = gdf.set_index("divide_id")
gdf.head()

,id,toid,mainstem,order,hydroseq,lengthkm,areasqkm,tot_drainage_areasqkm,has_divide,poi_id,vpuid,geometry
divide_id,,,,,,,,,,,,
cat-20466,wb-20466,nex-20467,2613576.0,1.0,20292,2.443793,4.304250,4.304250,True,19534,01,"MULTILINESTRING ((1873700.565 2684810.523, 187..."
cat-20474,wb-20474,nex-20475,2613597.0,1.0,20291,3.982518,4.829400,4.829400,True,None,01,"MULTILINESTRING ((1875378.459 2678583.977, 187..."
cat-20475,wb-20475,nex-20468,2613597.0,1.0,20290,1.006645,6.713101,11.542501,True,None,01,"MULTILINESTRING ((1875781.183 2682056.744, 187..."
cat-20473,wb-20473,nex-20467,2613587.0,1.0,20289,1.892781,4.486050,4.486050,True,None,01,"MULTILINESTRING ((1875252.558 2684695.906, 187..."
cat-20467,wb-20467,nex-20468,2613576.0,2.0,20288,1.616463,2.583900,22.916701,True,1193,01,"MULTILINESTRING ((1875278.618 2683667.5, 18754..."


In [ ]:
id_to_area = gdf["areasqkm"].to_dict()

divide_ids = np.unique(gdf.index)

zone_keys = [key for key in root.keys()]
zone_divide_ids = []
zone_runoff = []
for key in tqdm(zone_keys):
    zone_divide_ids.append(root[key]["divide_id"][:])
    zone_runoff.append(root[key]["Qr"][:])
all_divide_ids = np.concatenate(zone_divide_ids)
all_runoff = np.transpose(np.concatenate(zone_runoff))
del zone_divide_ids
del zone_runoff

100%|██████████| 96/96 [00:09<00:00, 10.08it/s]


In [ ]:
runoff_full_zone = np.zeros((all_runoff.shape[0], divide_ids.shape[0]))
indices = np.searchsorted(divide_ids, all_divide_ids)
runoff_full_zone[:, indices] = all_runoff

In [ ]:
areas = np.zeros_like(divide_ids, dtype=np.float64)
for idx, comid in enumerate(divide_ids):
    try:
        areas[idx] = id_to_area[comid]
    except KeyError as e:
        raise KeyError(f"problem finding {comid} in Areas Dictionary") from e
areas_array = areas * 1000 / 86400

streamflow_m3_s_data = runoff_full_zone * areas_array
streamflow_m3_s_data = np.nan_to_num(
    streamflow_m3_s_data, nan=1e-6, posinf=1e-6, neginf=1e-6
)
mask = streamflow_m3_s_data == 0
streamflow_m3_s_data[mask] = 1e-6

date_range = pd.date_range(
    start=cfg.create_streamflow.start_date,
    end=cfg.create_streamflow.end_date,
    freq="D",
)
    # data_array = xr.DataArray(
    #     data=streamflow_m3_s_data,
    #     dims=["time", "COMID"],  # Explicitly naming the dimensions
    #     coords={"time": date_range, "COMID": edge_comids},  # Adding coordinates
    # )
    # xr_dataset = xr.Dataset(
    #     data_vars={"streamflow": data_array},
    #     attrs={"description": "Streamflow -> MERIT Predictions"},
    # )
    # streamflow_path = Path(cfg.create_streamflow.data_store)
    # xr_dataset.to_zarr(streamflow_path, mode="w")

In [7]:
# datasets.append(xr.Dataset(
#     data_vars=dict(
#         Qr=(["divide_id", "time"], root[name]["Qr"][:])
#     ),
#     coords=dict(
#         divide_id=(["divide_id"], root[name]["divide_id"][:]),
#         time=(["time"], root[name]["time"][:])
#     ),
#     attrs=dict(description="Runoff outputs from dhbv2.0 at the HFv2.2 catchment scale"),
# ))